## Understanding ECB and CBC

Consider two messages, each of which owns n blocks: $M_1 = m_1||m_2||...||m_n$ and $M_2 = m'_1||m_2||...||m_n$, where $M_1$ and $M_2$ differ only in the first block. For instance, 

$M_1$ = "11335577 is my student ID. We will encrypt the message with different ways."  
$M_2$ = "22446688 is my student ID. We will encrypt the message with different ways."

Which of the following statements hold? Assume the same key is used for all encryption.

1) If $M_1$ and $M_2$ are encrypted using `Electronic Code Book (ECB)`, none of the cipher text blocks will repeat between each message.
2) If $M_1$ and $M_2$ are encrypted using `Cipher Block Chaining (CBC)`, but the same Initialization Vector (IV) is used for encrypting both messages, none of the cipher text blocks will repeat between each message.

In [1]:
M1 = b"11335577 is my student ID. We will encrypt the message in different ways."
M2 = b"22446688 is my student ID. We will encrypt the message in different ways."

In [2]:
!python -m pip install cryptography

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## Available modes in OpenSSL

aes-[128|192|256]-cbc  128/192/256 bit AES in CBC mode  
aes[128|192|256]       Alias for aes-[128|192|256]-cbc  
aes-[128|192|256]-cfb  128/192/256 bit AES in 128 bit CFB mode  
aes-[128|192|256]-cfb1 128/192/256 bit AES in 1 bit CFB mode  
aes-[128|192|256]-cfb8 128/192/256 bit AES in 8 bit CFB mode  
aes-[128|192|256]-ctr  128/192/256 bit AES in CTR mode  
aes-[128|192|256]-ecb  128/192/256 bit AES in ECB mode  
aes-[128|192|256]-ofb  128/192/256 bit AES in OFB mode

In [3]:
import random
import textwrap

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

SEED = 0
BLOCK_SIZE = 128

# seed
random.seed(0)

# generate random key and iv
KEY = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
IV = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
print(KEY.hex(), IV.hex())

c5d71484f8cf9bf4b76f47904730804b 9e3225a9f133b5dea168f4e2851f072f


In [4]:
# create padder that required by ECB and CBC modes, https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/
padder, unpadder = PKCS7(BLOCK_SIZE).padder(), PKCS7(BLOCK_SIZE).unpadder()
padded_data = padder.update(M1) + padder.finalize()
print(padded_data)
print(unpadder.update(padded_data) + unpadder.finalize())

padder, unpadder = PKCS7(BLOCK_SIZE).padder(), PKCS7(BLOCK_SIZE).unpadder()
padded_data = padder.update(M2) + padder.finalize()
print(padded_data)
print(unpadder.update(padded_data) + unpadder.finalize())

b'11335577 is my student ID. We will encrypt the message in different ways.\x07\x07\x07\x07\x07\x07\x07'
b'11335577 is my student ID. We will encrypt the message in different ways.'
b'22446688 is my student ID. We will encrypt the message in different ways.\x07\x07\x07\x07\x07\x07\x07'
b'22446688 is my student ID. We will encrypt the message in different ways.'


In [5]:
def encrypt_then_decrypt(plaintext, mode=modes.ECB()):
    cipher = Cipher(algorithms.AES(KEY), mode)
    encryptor, decryptor = cipher.encryptor(), cipher.decryptor()
    padder, unpadder = PKCS7(128).padder(), PKCS7(128).unpadder()

    # encrypt
    if len(plaintext) % 16 != 0:
        padded_data = padder.update(plaintext) + padder.finalize()
    else:
        padded_data = plaintext

    ciphered_data = encryptor.update(padded_data) + encryptor.finalize()

    # decrypt
    decipered_data = decryptor.update(ciphered_data) + decryptor.finalize()
    if len(plaintext) % 16 != 0:
        unpadded_data = unpadder.update(decipered_data) + unpadder.finalize()
        assert plaintext == unpadded_data

    return ciphered_data

In [6]:
color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

def colorize(string, color, bold=False, highlight=False):
    """
    Colorize a string.

    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)

def visual_hex_diff(a,b):
    a, b = textwrap.wrap(a.hex(), 16), textwrap.wrap(b.hex(), 16)
    SEP = "   |   "
    print(" ", "ciphered M1".ljust(16), "ciphered M2", sep=SEP)
    for i, (m1, m2) in enumerate(zip(a, b)):
        color = "red" if m1 != m2 else "green"
        print(i, m1, colorize(m2, color), sep=SEP)

In [7]:
ciphered_M1 = encrypt_then_decrypt(M1, modes.ECB())
ciphered_M2 = encrypt_then_decrypt(M2, modes.ECB())
visual_hex_diff(ciphered_M1, ciphered_M2)

    |   ciphered M1        |   ciphered M2
0   |   f9ac8dae3bfef744   |   9159269855625cc3
1   |   880c3c13ee28eeb2   |   939c0187877ec1e0
2   |   e41e2fadaa393921   |   e41e2fadaa393921
3   |   5521cdf80420a7d9   |   5521cdf80420a7d9
4   |   e5ae86391ea0185b   |   e5ae86391ea0185b
5   |   be9fdad51605d41b   |   be9fdad51605d41b
6   |   56c843974849eef2   |   56c843974849eef2
7   |   81876d73acd99d70   |   81876d73acd99d70
8   |   0d7a24960697c96b   |   0d7a24960697c96b
9   |   fe8d61392693c8d7   |   fe8d61392693c8d7


In [8]:
ciphered_M1 = encrypt_then_decrypt(M1, modes.CBC(IV))
ciphered_M2 = encrypt_then_decrypt(M2, modes.CBC(IV))
visual_hex_diff(ciphered_M1, ciphered_M2)

    |   ciphered M1        |   ciphered M2
0   |   510a1f951e74d2bd   |   04d75e345474efd6
1   |   918c607c59ecc708   |   c93c7b2767d03a2c
2   |   def5a0a286d78a6f   |   fb32417bf8a80ab3
3   |   b568eefc924391a9   |   bd46274130510725
4   |   8173cc12217f0a39   |   c5aa36451cb2f9a4
5   |   2a738219ed52c601   |   22ce4aace2daed57
6   |   6534860559b1c8aa   |   a3241f83268b2fa3
7   |   6a82f0eba8bd5793   |   359ab8d3313eed73
8   |   a4a8da02beaeea97   |   fc69795dc1970d57
9   |   931c5b3ba98e52b1   |   974a15b3cbda9da3
